In [6]:
import pandas as pd
import json
import statistics
import numpy as np
import datetime as dt
import iexfinance as iexf
import pyprind as bar

In [7]:
start = dt.date(2018,5,15)
end = dt.date(dt.date.today().year, dt.date.today().month,dt.date.today().day-1)
end_t = dt.datetime(end.year,end.month,end.day-1, 4,0,0)

#### Yahoo! Finance Data

In [8]:
start_utc = int(dt.datetime(2018,5,15,4,0,0).timestamp())
end_utc = int(end_t.timestamp())

yahoo_url = "https://finance.yahoo.com/quote/{0}/history?period1={1}&period2={2}&interval=1d&filter=history&frequency=1d"
url_tail = "/history?p=%5"

yahoo_symbbol = ["^DJA",
                "^GSPC",
                "EUXL"]

yahoo_description = {"DJA": "Dow Jones Industrial Average",
                     "GSPC":"Standard and Poors 500",
                     "EUXL":"Direxion Daily Euro Stoxx 50 Bull 3x Shares"}

In [12]:
progress = bar.ProgBar(len(yahoo_symbbol), monitor=True, title="Scraping Yahoo! Finance Historical Prices")
for sym in yahoo_symbbol:
    temp = pd.read_html(yahoo_url.format(sym,start_utc,end_utc))
    df = temp[0][:-1].copy()
    df["Symbol"] = sym.replace("^","")
    globals()["yahoo_%s" %sym.replace("^","")] = df
    progress.update()

Scraping Yahoo! Finance Historical Prices
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


#### IEX Financial Data

In [13]:
iex_symbol = {"DIA":"DJA", "SPY":"GSPC", "EUXL":"EUXL"}
iex_name = ["Dow Jones Industrial Average", "Standard and Poors 500", "Direxion Daily Euro Stoxx 50 Bull 3x Shares"]

iex_col_data = ["Date","Open","High","Low","Close","Volume","Symbol"]
iex_col_dtypes = {
    "Date":"datetime64[ns]",
    "Open":"float64",
    "High":"float64",
    "Low":"float64",
    "Close":"float64",
    "Volume":"float64",
    "Symbol":"object"}

In [14]:
for stock in iex_symbol.keys():
    globals()["iex_%s" %stock] = iexf.get_historical_data(stock, start=start, end=end)
    temp_date = []
    temp_open = []
    temp_high = []
    temp_low = []
    temp_close = []
    temp_vol = []
    temp_symbol = []

    temp_keys = globals()["iex_%s" %stock][stock]
    for key in temp_keys.keys():
        temp_date.append(key)
        temp_open.append(temp_keys[key]["open"])
        temp_high.append(temp_keys[key]["high"])
        temp_low.append(temp_keys[key]["low"])
        temp_close.append(temp_keys[key]["close"])
        temp_vol.append(temp_keys[key]["volume"])
        temp_symbol.append(iex_symbol[stock])
    globals()["iex_%s" %stock] = pd.DataFrame(np.column_stack([temp_date,temp_open, temp_high,temp_low,temp_close,temp_vol,temp_symbol]),
                          columns=iex_col_data).astype(iex_col_dtypes)


##### Yahoo! Finance DataFrames with Date Index

In [15]:
yahoo_DJA_df = yahoo_DJA.set_index("Date")
yahoo_DJA_df.sort_index(ascending=True, inplace=True)
yahoo_GSPC_df = yahoo_GSPC.set_index("Date")
yahoo_GSPC_df.sort_index(ascending=True, inplace=True)
yahoo_EUXL_df = yahoo_EUXL.set_index("Date")
yahoo_EUXL_df.sort_index(ascending=True, inplace=True)

In [16]:
yahoo_DJA_df.head()

,Open,High,Low,Close*,Adj Close**,Volume,Symbol
Date,,,,,,,
"Jun 01, 2018",8270.94,8347.58,8231.10,8290.68,8290.68,3.684130e+09,DJA
"Jun 04, 2018",8314.67,8386.66,8252.34,8298.82,8298.82,3.376510e+09,DJA
"Jun 05, 2018",8297.31,8335.83,8224.04,8278.17,8278.17,3.517790e+09,DJA
"Jun 06, 2018",8290.25,8377.35,8225.07,8336.79,8336.79,3.651640e+09,DJA
"Jun 07, 2018",8348.03,8428.50,8294.84,8361.96,8361.96,3.711330e+09,DJA


In [17]:
yahoo_GSPC_df.head()

,Open,High,Low,Close*,Adj Close**,Volume,Symbol
Date,,,,,,,
"Jun 01, 2018",2718.70,2736.93,2718.70,2734.62,2734.62,3.684130e+09,GSPC
"Jun 04, 2018",2741.67,2749.16,2740.54,2746.87,2746.87,3.376510e+09,GSPC
"Jun 05, 2018",2748.46,2752.61,2739.51,2748.80,2748.80,3.517790e+09,GSPC
"Jun 06, 2018",2753.25,2772.39,2748.46,2772.35,2772.35,3.651640e+09,GSPC
"Jun 07, 2018",2774.84,2779.90,2760.16,2770.37,2770.37,3.711330e+09,GSPC


In [18]:
yahoo_EUXL_df.head()

,Open,High,Low,Close*,Adj Close**,Volume,Symbol
Date,,,,,,,
"Jun 01, 2018",23.42,23.44,23.33,23.44,23.07,2300,EUXL
"Jun 04, 2018",23.89,23.89,23.89,23.89,23.52,400,EUXL
"Jun 05, 2018",23.69,23.70,23.41,23.65,23.29,1300,EUXL
"Jun 06, 2018",24.05,24.42,24.05,24.42,24.04,1100,EUXL
"Jun 07, 2018",24.34,24.34,23.78,23.78,23.41,1300,EUXL


##### IEX Financial DataFrames with Date Index

In [19]:
iex_DJA_df = iex_DIA.set_index("Date")
iex_GSPC_df = iex_SPY.set_index("Date")
iex_EUXL_df = iex_EUXL.set_index("Date")

In [20]:
iex_DJA_df.head()

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,247.0021,247.1116,245.4289,246.2454,4755865.0,DJA
2018-05-16,246.1159,247.2112,245.9078,246.9025,2034402.0,DJA
2018-05-17,246.7333,247.8212,245.8172,246.6735,2058226.0,DJA
2018-05-18,246.6010,247.1602,246.0518,246.6310,2408831.0,DJA
2018-05-21,248.8477,250.2955,248.8377,249.5666,4060299.0,DJA


In [21]:
iex_GSPC_df.head()

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,270.3763,270.3962,268.8232,269.8885,87036136.0,GSPC
2018-05-16,269.9283,271.5410,269.8984,271.0234,53942631.0,GSPC
2018-05-17,270.7247,272.0089,269.9183,270.7944,56536400.0,GSPC
2018-05-18,270.4061,270.8143,269.7192,270.1174,64367959.0,GSPC
2018-05-21,271.7899,272.7556,270.1385,272.1483,58025875.0,GSPC


In [22]:
iex_EUXL_df.head()

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,26.0672,26.0672,26.0672,26.0672,501.0,EUXL
2018-05-16,25.5947,25.9065,25.5947,25.9065,925.0,EUXL
2018-05-17,26.2788,26.2788,26.2788,26.2788,1081.0,EUXL
2018-05-18,25.9150,25.9884,25.8998,25.8998,1159.0,EUXL
2018-05-21,25.9983,26.1558,25.9983,26.1166,4620.0,EUXL
